In [1]:
from __future__ import print_function, division
import pyigrf
import numpy as np
import datetime

In [2]:
# Geodetic to Geocentric coordinates transformation
# WGS84 constants
# reference:
# http://earth-info.nga.mil/GandG/publications/tr8350.2/wgs84fin.pdf
a_WGS=6378.137
flatness = 1./298.257223563  # flatenning
b_WGS=a_WGS*(1.-flatness)    # WGS polar radius (semi-minor axis) in km
eccentricity=np.sqrt(a_WGS**2-b_WGS**2)/a_WGS

def geod2geoc(lon,geodlat,h):
    # returns geocentric xyz coordinates (ECEF) in km of a target with
    # latitude       geodlat (rad) --- geodetic
    # longitude      lon (rad)
    # height         h (km above local ellipsoid)
    n=a_WGS / np.sqrt(1.-flatness*(2.-flatness) * np.sin(geodlat)**2.)
    # cartesian geocentric coordinates wrt Greenwich
    x=(n+h)*np.cos(geodlat)*np.cos(lon)
    y=(n+h)*np.cos(geodlat)*np.sin(lon)
    z=(n*(1.-eccentricity**2.)+h)*np.sin(geodlat)   
    
    p   = np.sqrt(x**2. + y**2.)
    geoclat = np.arctan2(z,p)        # geocentric latitude (theta)
    
    return x,y,z,geoclat

## B @ 100 km above PFISR on Feb 18, 2021:
https://amisr.com/amisr/about/about_pfisr/pfisr-specs/

Latitude, Longitude, Altitude: 65.12992 N, 147.47104 W, 213 m

In [3]:
def datetime2years(dt0):
    daysinyear = (dt0 - datetime.datetime(dt0.year,1,1)).total_seconds()/24/3600
    totaldaysinyear = datetime.datetime(dt0.year,12,31).timetuple().tm_yday
    return dt0.year + daysinyear/totaldaysinyear

geod_lat = 65.12992
lon = -147.47104
geod_ht = 100   #km above WGS-84 ellipsoide
print("geodetic coordinates: latitude:%g deg, longitude:%g deg, altitude:%g km"%(
    geod_lat,lon,geod_ht))
deg = 180/np.pi
x,y,z,geoc_lat = geod2geoc(lon/deg, geod_lat/deg, geod_ht) # convert to geocentric
ht_igrf = np.sqrt(x**2 + y**2 + z**2) - pyigrf.a_igrf # height above IGRF sphere with radius 6371.2 km
dt0 = datetime.datetime(2021,2,18,12,12,12)
year = datetime2years(dt0)

print("geocentric coordinates: latitude:%g deg, longitude:%g deg, IGRF altitude:%g km"%(
    geoc_lat*deg,lon,ht_igrf))
[Bn,Be,Bd,B] = pyigrf.igrf_B(year, ht_igrf, lon, geoc_lat*deg)
print("B = %g nT: %g nT northwards, %g nT eastwards, %g nT downwards"%(
         B,Bn,Be,Bd))

geodetic coordinates: latitude:65.1299 deg, longitude:-147.471 deg, altitude:100 km
geocentric coordinates: latitude:64.985 deg, longitude:-147.471 deg, IGRF altitude:89.3606 km
B = 53957.2 nT: 11241.4 nT northwards, 3357.46 nT eastwards, 52666.3 nT downwards
